<a href="https://colab.research.google.com/github/brooketalcott/ML-Practice/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
def rem_outliers(dataframe,orig,col,min_val):
  df = dataframe.copy()
  df = (df[df[col]
             .isin(orig[col]
                   .value_counts()[lambda x: x >= min_val]
                   .index
                   .to_list()
                   )
             ]
        )
  return df

df = pd.merge(df_ratings,df_books,on='isbn')

df_r = (df
        .pipe(rem_outliers,df_ratings,'user',200)
        .pipe(rem_outliers,df_ratings,'isbn',100)
        )

# stats = df_r.describe()['rating']
# df_r['rating'] = (df_r['rating']-stats['min'])/(stats['max']-stats['min'])
# df_r['rating']

In [ ]:
df_r_pivot = df_r.pivot(index='isbn',columns='user',values='rating').fillna(0)
df_r_matrix = csr_matrix(df_r_pivot.values)


In [ ]:
model = NearestNeighbors(metric='cosine')
model.fit(df_r_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
def get_isbn(title):
  isbn = df_books.query('title == @title').isbn.values[0]
  return isbn

In [ ]:
isbn = get_isbn("Where the Heart Is (Oprah's Book Club (Paperback))")
book_rec = df_r_pivot[df_r_pivot.index.isin([isbn])]
dist,inst = model.kneighbors(
    book_rec.values.reshape(1,-1),n_neighbors=6)
distances = dist.flatten()
instances = inst.flatten()
details = [
         [df_books.query(
             f'isbn == "{df_r_pivot.index[instances[i]]}"'
             ).title.values[0]
          ,round(distances[i],2)]
         for i in range(len(distances))
]

reccommends = [details[0][0]] + [details[2:]]
reccommends[1].sort(key=lambda x: x[1],reverse=True)
reccommends

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.8],
  ['I Know This Much Is True', 0.77],
  ['The Surgeon', 0.77],
  ['The Weight of Water', 0.77]]]

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  isbn = get_isbn(book)
  book_rec = df_r_pivot[df_r_pivot.index.isin([isbn])]
  dist,inst = model.kneighbors(
      book_rec.values.reshape(1,-1),n_neighbors=6)
  distances = dist.flatten()
  instances = inst.flatten()
  details = [
          [df_books.query(
              f'isbn == "{df_r_pivot.index[instances[i]]}"'
              ).title.values[0]
            ,round(distances[i],2)]
          for i in range(len(distances))
  ]

  reccommends = [details[0][0]] + [details[2:]]
  reccommends[1].sort(key=lambda x: x[1],reverse=True)

  return reccommends

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

You passed the challenge! 🎉🎉🎉🎉🎉
